In [6]:
companies = ['gazp', 'sber', 'vtbr','gmkn','poly','yndx','lkoh','alrs','nlmk','rosn','chmf','magn','afks','rual','five','nvtk','tatn','mgnt','mail','mtss','irao','dsky','plzl','moex','aflt','phor','tcsg','pikk','sngsp','mtlr','sberp','mvid','pogr','sngs','tatnp','sibn','fees','hydr','rasp','agro','belu','upro','banep','rsti','qiwi']
not_stock_prices_companies = ['sber', 'vtbr','moex','five', 'mail', 'tcsg', 'pogr', 'agro', 'banep', 'qiwi', 'sngsp', 'sberp','tatnp']

params = ['Всего выпущено обыкновенных акций',
       'Количество акций доступных к купле-продаже', 'Количество сотрудников',
       'Количество акционеров', 'Цена/Прибыль', 'Цена/Продажи',
       'Цена/Движение денежных средств', 'Цена/балансовая стоимость',
       'Стоимость компании', 'Стоимость компании/EBITDA',
       'Прибыль на общ. сумму активов, %', 'Доход на капитал, %',
       'Доход на инвестированный капитал, %', 'Валовая рентабельность %',
       'Операционная рентабельность %', 'Рентабельность по EBITDA %',
       'Чистая рентабельность %', 'Быстрая ликвидность',
       'Коэффициент текущей ликвидности', 'Оборачиваемость запасов',
       'Оборачиваемость активов', 'Задолженность/Активы',
       'Задолженность/Капитал', 'Долгосрочная задолж./совокупные активы', 'Mean_stock_price_with_inflation']

params = ['Всего выпущено обыкновенных акций',
       'Цена/Прибыль', 'Цена/Продажи',
       'Цена/Движение денежных средств', 'Цена/балансовая стоимость',
       'Стоимость компании', 'Стоимость компании/EBITDA',
       'Прибыль на общ. сумму активов, %', 'Доход на капитал, %',
       'Доход на инвестированный капитал, %', 'Валовая рентабельность %',
       'Операционная рентабельность %', 'Рентабельность по EBITDA %',
       'Чистая рентабельность %', 'Быстрая ликвидность',
       'Коэффициент текущей ликвидности', 'Оборачиваемость запасов',
       'Оборачиваемость активов', 'Задолженность/Активы',
       'Задолженность/Капитал', 'Долгосрочная задолж./совокупные активы', 'Mean_stock_price_with_inflation']
import pandas as pd
delta_frame = pd.DataFrame(columns = params)
def get_from_json_fund_params(name_company):
    import pandas as pd
    import os
    directory = r'C:\Users\pc\Downloads\json_date'
    files = os.listdir(directory)
    y_html_files = list(filter(lambda x: x.endswith('y.html'), files))
    #print(y_html_files)
    file_path = fr'C:\Users\pc\Downloads\json_date\{name_company.upper()}_y.html'
    if file_path[32:] not in y_html_files:
        return
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_rows', None)
    from bs4 import BeautifulSoup
    with open(file_path, 'r', encoding='utf8') as f:
        soup = BeautifulSoup(f, 'lxml')
        items_date_1 = soup.find_all("div", class_="wrap-25PNPwRV")
        items_titles_1 = soup.find_all("span", class_="title-jKD0Exn- apply-overflow-tooltip")
    items_date_1_1 = []
    for item in items_date_1:
        items = item.get_text()
        items_date_1_1.append(items)
    #print(items_date_1_1)
    items_titles_1_1 = []
    for item in items_titles_1:
        items = item.get_text()
        items_titles_1_1.append(items)
    #print(len(items_titles_1_1))
    item_date = [[] for i in range(len(items_titles_1) + 1)]
    ind, num_list = 1, 0
    for i in items_date_1_1:
        item_date[num_list].append(i)
        ind += 1
        if ind % 9 == 0:
            num_list += 1
            ind = 1
    #print(item_date)
    data = pd.DataFrame(item_date[1:], columns=item_date[0], index=items_titles_1_1)
    #print(data)
    q_html_files = list(filter(lambda x: x.endswith('q.html'), files))
    file_path = fr'C:\Users\pc\Downloads\json_date\{name_company.upper()}_q.html'
    if file_path[32:] in q_html_files:
        data.drop(['2020', 'Текущ. цена'], axis=1, inplace=True)
    data = data.T
    for i in data.columns:
        for j in range(len(data[i])):
            if data[i][j] is not None and len(data[i][j]) > 1:
                if data[i][j][-2] == "K" or data[i][j][-2] == "B" or data[i][j][-2] == "M":
                    if data[i][j][1] == '−':
                        data[i][j] = float('-' + data[i][j][2:-2])
                    else:
                        data[i][j] = float(data[i][j][1:-2])
                    #print(type(data[i][j]))
                elif data[i][j][1] == '−':
                    data[i][j] = float('-'+data[i][j][2:-1])
                else:
                    data[i][j] = float(data[i][j][1:-1])
                #print(data[i][j])
            elif data[i][j] == '—':
                data[i][j] = None
            #print(data[i][j], len(data[i][j]), data[i][j][1:-1])
    q_html_files = list(filter(lambda x: x.endswith('q.html'), files))
    file_path = fr'C:\Users\pc\Downloads\json_date\{name_company.upper()}_q.html'
    if file_path[32:] not in q_html_files:
        import investpy
        data_stock = investpy.get_stock_historical_data(stock=name_company, country='russia', from_date='01/01/2013',
                                                        to_date='05/11/2021')
        data_stock_y = data_stock[:'20220101']
        d_years = data_stock_y.resample('Y')['Close'].mean()
        df = d_years
        # print(df)
        import calendar
        voc_mean_stock_price = {}
        for ind, i in enumerate(df.index):
            if str(i.year) in data.index and ind != len(df.index) - 1:
                # print (i)
                voc_mean_stock_price[str(i.year)] = df[ind]
            else:
                voc_mean_stock_price[data.index[-1]] = df[
                    pd.Timestamp(i.year, i.quarter * 3, calendar.monthrange(i.year, i.quarter * 3)[1])]
        #print (voc_mean_stock_price)
        #print(data)
        ser_mean_stock_price = pd.Series(voc_mean_stock_price, name='Mean_stock_price')
        data_full_with_stock_price = data.T.append(ser_mean_stock_price, ignore_index=False)
        #print (data_full_with_stock_price)
        import numpy as np
        inflation = np.array([11.36, 12.91, 5.38, 2.52, 4.27, 3.05, 4.91, 5.32])[::-1]
        inflation = np.cumsum(inflation)[::-1]
        # print(data_full_with_stock_price.loc['Mean_stock_price'])
        stock_price_with_inflation = data_full_with_stock_price.loc['Mean_stock_price'] * (1 + inflation / 100)
        stock_price_with_inflation.name = 'Mean_stock_price_with_inflation'
        data_full_with_stock_price_with_inflation = data_full_with_stock_price.append(stock_price_with_inflation)

    else:
        with open(file_path, 'r', encoding='utf8') as f:
            soup = BeautifulSoup(f, 'lxml')
            items_date_1 = soup.find_all("div", class_="wrap-25PNPwRV")
            items_titles_1 = soup.find_all("span", class_="title-jKD0Exn- apply-overflow-tooltip")

        items_date_q = []
        for item in items_date_1:
            items = item.get_text()
            items_date_q.append(items)
        # print(items_date_q)

        items_titles_q = []
        for item in items_titles_1:
            items = item.get_text()
            items_titles_q.append(items)
        #print(len(items_titles_q))

        item_date = [[] for i in range(24)]
        ind, num_list = 1, 0
        for i in items_date_q:
            item_date[num_list].append(i)
            ind += 1
            if ind % 9 == 0:
                num_list += 1
                ind = 1
        import pandas as pd
        #print(len(item_date))
        none_3_cols = [[None for i in range(len(item_date[0]))] for j in range(3)]
        #print(none_3_cols)
        data_q_rows = [[0 for i in range(len(item_date[0]))] for j in range(24)]
        data_q_rows[0] = item_date[1]
        for m in range(3):
            data_q_rows[1+m] = none_3_cols[m]
        for k in range(20):
            data_q_rows[4+k] = item_date[2+k]
            #print(item_date[21])
        #print(len(data_q_rows))
        data_q = pd.DataFrame(data_q_rows, columns=item_date[0], index=items_titles_1_1)
        #print(data_q)
        quartiles_index = data_q.columns
        # print(quartiles_index)
        data_q = data_q.T
        for i in data_q.columns:
            for j in range(len(data_q[i])):
                if data_q[i][j] is not None and len(data_q[i][j]) > 1:
                    if data_q[i][j][-2] == "K" or data_q[i][j][-2] == "B" or data_q[i][j][-2] == "M":
                        if data_q[i][j][1] == '−':
                            data_q[i][j] = float('-' + data_q[i][j][2:-2])
                        else:
                            data_q[i][j] = float(data_q[i][j][1:-2])
                        # print(type(data_q[i][j]))
                    elif data_q[i][j][1] == '−':
                        data_q[i][j] = float('-' + data_q[i][j][2:-1])
                    else:
                        data_q[i][j] = float(data_q[i][j][1:-1])
                    # print(data_q[i][j])
                elif data_q[i][j] == '—':
                    data_q[i][j] = None
                # print(data_q[i][j], len(data_q[i][j]), data_q[i][j][1:-1])
        data_full = pd.concat([data.T, data_q.T], axis=1)
        if 'Q4 19' in data_full.columns:
            data_full.drop('Q4 19', axis=1, inplace=True)
        #print(data_full)
        import investpy
        data_stock = investpy.get_stock_historical_data(stock=name_company, country='russia', from_date='01/01/2013',
                                                        to_date='05/11/2021')
        data_stock_y = data_stock[:'20200101']
        d_years = data_stock_y.resample('Y')['Close'].mean()
        data_stock_q = data_stock['20200101':]
        d_quarts = data_stock_q.resample('Q')['Close'].mean()
        df = d_years.append(d_quarts)
        # print(df)
        import calendar
        voc_mean_stock_price = {}
        for ind, i in enumerate(df.index):
            if str(i.year) in data.index:
                # print (i)
                voc_mean_stock_price[str(i.year)] = df[ind]
            elif i.year > 2018 and ind != len(df.index) - 1 and (
                    'Q' + str(i.quarter) + ' ' + str(i.year)[2:]) in quartiles_index:
                # print('Q' + str(i.quarter) + ' ' + str(i.year)[2:], ('Q' + str(i.quarter) + ' ' + str(i.year)[2:]) in quartiles_index)
                voc_mean_stock_price['Q' + str(i.quarter) + ' ' + str(i.year)[2:]] = df[
                    pd.Timestamp(i.year, i.quarter * 3, calendar.monthrange(i.year, i.quarter * 3)[1])]
            else:
                voc_mean_stock_price[data_full.columns[-1]] = df[
                    pd.Timestamp(i.year, i.quarter * 3, calendar.monthrange(i.year, i.quarter * 3)[1])]
        # print (voc_mean_stock_price)

        ser_mean_stock_price = pd.Series(voc_mean_stock_price, name='Mean_stock_price')
        #print(ser_mean_stock_price)
        data_full_with_stock_price = data_full.append(ser_mean_stock_price, ignore_index=False)
        #print (data_full_with_stock_price)
        import numpy as np
        if len(data_full_with_stock_price.columns) == 14:
            inflation = np.array([11.36, 12.91, 5.38, 2.52, 4.27, 3.05, 1.28, 1.32, 0.24, 1.97, 2.11, 2.01, 1.08, 2.3])[::-1]
        else:
            inflation = np.array([11.36, 12.91, 5.38, 2.52, 4.27, 3.05, 1.28, 1.32, 0.24, 1.97, 2.11, 2.01, 2.3])[::-1]
        inflation = np.cumsum(inflation)[::-1]
        # print(data_full_with_stock_price.loc['Mean_stock_price'])
        stock_price_with_inflation = data_full_with_stock_price.loc['Mean_stock_price'] * (1 + inflation / 100)
        stock_price_with_inflation.name = 'Mean_stock_price_with_inflation'
        data_full_with_stock_price_with_inflation = data_full_with_stock_price.append(stock_price_with_inflation)
    data_full_with_stock_price_with_inflation.drop('Mean_stock_price', axis = 0, inplace = True)
    data_full_with_stock_price_with_inflation = data_full_with_stock_price_with_inflation.T
    #for i in data_full_with_stock_price_with_inflation.columns:
        #if len(data_full_with_stock_price_with_inflation) <= 2 * data_full_with_stock_price_with_inflation[i].isna().sum():
            #data_full_with_stock_price_with_inflation.drop(i, axis = 1, inplace = True)
    data_full_with_stock_price_with_inflation.drop(columns = ['Количество акций доступных к купле-продаже', 'Количество сотрудников',
       'Количество акционеров'], inplace = True)
    data_full_with_stock_price_with_inflation.fillna(method = 'ffill', inplace = True)
    data_full_with_stock_price_with_inflation.fillna(method = 'bfill', inplace = True)
    data_full_with_stock_price_with_inflation.fillna(value = 1, inplace = True)
    #print(data_full_with_stock_price_with_inflation)
    #return data_full_with_stock_price_with_inflation
    #print(data_full_with_stock_price_with_inflation.T.index)
    for i in range(len(data_full_with_stock_price_with_inflation.index) - 1):
        row_delta = 100 * (data_full_with_stock_price_with_inflation.iloc[i+1] - data_full_with_stock_price_with_inflation.iloc[i]) / data_full_with_stock_price_with_inflation.iloc[i] 
        #row_delta = data_full_with_stock_price_with_inflation.iloc[i+1] - data_full_with_stock_price_with_inflation.iloc[i]
        delta_frame.loc[i] = row_delta
        #print(row_delta)
    #return delta_frame
    delta_frame['ticker'] = name_company
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_rows', None)
    return delta_frame
    #delta_frame.to_csv(r'C:\Users\pc\Downloads\json_date\test.txt', header=None, index=None, sep=',', mode='a')
    #data_full_with_stock_price_with_inflation.to_csv(r'C:\Users\pc\Downloads\json_date\container.txt', header=None, index=None, sep=',', mode='a')


#for i in companies:
    #if i in not_stock_prices_companies:
        #continue
    #get_from_json_fund_params(i)

#for i in ['sber', 'vtbr', 'moex']:
    #print(get_from_json_fund_params(i))
#print(get_from_json_fund_params('moex'))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
for i in ['sber', 'vtbr', 'moex']:
    print(get_from_json_fund_params(i))
#print(pd.read_csv(r'C:\Users\pc\Downloads\json_date\container.txt', header = None))

    Всего выпущено обыкновенных акций  Цена/Прибыль  Цена/Продажи  \
0                            0.088928    150.245098     32.075472   
1                            0.000000    -29.285015     80.000000   
2                           -0.088849     -6.509695     34.920635   
3                           -0.088928    -24.444444    -22.352941   
4                           -0.044504     24.313725     27.272727   
5                            0.000000    -18.296530    -25.595238   
6                            0.000000     20.656371      7.200000   
7                            0.000000      8.800000     12.686567   
8                           -0.044524     21.323529     17.880795   
9                            0.133630    -14.303030      2.247191   
10                           0.000000     -8.628006      0.549451   
11                           0.000000      2.321981     18.032787   
12                           0.000000      8.774584      8.796296   

    Цена/Движение денежных средст

    Всего выпущено обыкновенных акций  Цена/Прибыль  Цена/Продажи  \
0                          -62.234663      0.000000    -61.139896   
1                            0.027711    -97.097867      8.000000   
2                            0.012691    -76.221358    -46.913580   
3                            0.196759    -52.727273    -27.906977   
4                            0.051661     21.923077     45.161290   
5                            0.000000    -28.706625    -37.777778   
6                            0.000000     26.548673     21.428571   
7                            0.000000     25.524476     -5.882353   
8                            0.011500    110.027855     21.875000   
9                            0.000000    -30.238727     10.256410   
10                           0.000000    -38.593156      4.651163   
11                           0.000000    -21.362229      0.000000   
12                          -0.000309      0.000000      0.000000   

    Цена/Движение денежных средст

    Всего выпущено обыкновенных акций  Цена/Прибыль  Цена/Продажи  \
0                            0.450450    -10.744811      2.534562   
1                            0.896861     53.077975     45.842697   
2                            0.000000      7.953530     -3.697997   
3                            0.000000    -23.509934    -27.360000   
4                            0.000000     30.194805     16.519824   
5                            0.000000    -20.781380    -13.610586   
6                            0.000000     11.962225     14.660832   
7                            0.000000     28.584817     28.816794   
8                            0.444444      4.227405      5.925926   
9                            0.000000      5.104895      7.692308   
10                           0.000000     -2.395210     -3.376623   
11                           0.884956      1.840491      1.344086   
12                          -0.000309      0.000000      0.000000   

    Цена/Движение денежных средст

In [7]:
gazp = get_from_json_fund_params('gazp')
for i in gazp.columns:
    if len(gazp) <= 2 * gazp[i].isna().sum():
        gazp.drop(i, axis = 1, inplace = True)
gazp.fillna(method = 'ffill', inplace = True)
gazp

,Всего выпущено обыкновенных акций,Цена/Прибыль,Цена/Продажи,Цена/Движение денежных средств,Цена/балансовая стоимость,Стоимость компании,Стоимость компании/EBITDA,"Прибыль на общ. сумму активов, %","Доход на капитал, %","Доход на инвестированный капитал, %",Валовая рентабельность %,Операционная рентабельность %,Рентабельность по EBITDA %,Чистая рентабельность %,Быстрая ликвидность,Коэффициент текущей ликвидности,Оборачиваемость запасов,Оборачиваемость активов,Задолженность/Активы,Задолженность/Капитал,Долгосрочная задолж./совокупные активы,Mean_stock_price_with_inflation,ticker
0,0.000000,-78.897137,-5.454545,-0.609756,-3.225806,11.854625,1.214575,339.639640,364.457831,345.323741,-3.653169,1.614060,1.205479,353.448276,0.000000,1.075269,-5.197505,-2.631579,11.111111,18.518519,6.666667,-3.873406,gazp
1,-3.698869,-7.788945,13.461538,42.331288,3.333333,4.240528,24.400000,14.754098,13.878080,14.539580,-15.121060,-24.355807,-16.053059,21.292776,-12.666667,-10.638298,8.552632,-5.405405,-15.000000,-21.875000,-12.500000,-9.709550,gazp
2,0.000000,10.081744,-23.728814,12.931034,-19.354839,-2.155580,-12.218650,-27.500000,-28.359909,-26.657264,9.822390,5.786281,4.095453,-29.843260,-20.610687,-20.238095,2.828283,2.857143,5.882353,8.000000,-7.142857,-13.581951,gazp
3,0.000000,-42.574257,-8.888889,-15.648855,0.000000,21.495183,-16.117216,83.004926,85.691574,82.884615,23.180593,25.231584,12.360595,58.266309,28.846154,26.865672,-4.518664,16.666667,0.000000,3.703704,23.076923,11.999816,gazp
4,6.823317,106.896552,82.926829,61.085973,72.000000,41.148051,115.720524,-24.764468,-24.828767,-25.552050,-23.572707,-43.008101,-29.638820,-11.349520,-14.925373,-11.176471,4.526749,-14.285714,5.555556,0.000000,-6.250000,36.231804,gazp
5,0.000000,53.750000,-20.000000,-20.505618,-30.232558,-9.667890,6.275304,-55.456172,-54.897494,-55.932203,-12.259240,-28.121137,-9.482759,-142.547771,8.771930,7.284768,10.433071,-11.111111,10.526316,14.285714,20.000000,1.431378,gazp
6,0.000000,52.168022,18.333333,45.229682,10.000000,3.130335,34.476190,-26.907631,-26.262626,-26.282051,-30.080095,-138.693035,-46.060606,-291.916168,-13.709677,-8.641975,-14.081996,-6.250000,0.000000,0.000000,0.000000,-12.421596,gazp
7,0.000000,0.000000,-9.859155,-20.924574,-12.121212,2.663126,12.747875,-115.934066,-115.753425,-115.652174,24.310564,-176.666667,41.171750,-240.171607,-13.084112,-5.405405,-10.373444,-6.666667,9.523810,12.500000,11.111111,-4.955085,gazp
8,0.000000,236.153161,25.000000,-14.153846,24.137931,6.574174,-3.517588,-303.448276,-306.521739,-305.555556,18.020478,310.434783,37.862422,-196.994992,15.053763,2.857143,5.092593,0.000000,-4.347826,0.000000,0.000000,-2.445440,gazp
9,0.000000,-79.523179,-2.500000,3.942652,2.777778,2.204738,-18.880208,400.000000,408.421053,400.000000,23.713129,62.570621,32.412371,11.761331,23.364486,12.500000,20.704846,3.571429,0.000000,0.000000,0.000000,21.586369,gazp


In [16]:
for i in range(len(gazp.index) - 1):
        row_delta = 100 * gazp.iloc[i] / gazp.iloc[i+1] - 100
        #row_delta = gazp.iloc[i+1] - gazp.iloc[i]
        delta_frame.loc[i] = row_delta
        print(row_delta)
print(delta_frame)

Всего выпущено обыкновенных акций               0.00
Количество акций доступных к купле-продаже      0.00
Количество сотрудников                          0.75
Количество акционеров                           0.00
Цена/Прибыль                                  -45.20
Цена/Продажи                                   -1.06
Цена/Движение денежных средств                 -6.46
Цена/балансовая стоимость                      -1.99
Стоимость компании                           -193.78
Стоимость компании/EBITDA                     -11.14
Прибыль на общ. сумму активов, %                2.18
Доход на капитал, %                            26.89
Доход на инвестированный капитал, %             3.32
Валовая рентабельность %                        5.60
Операционная рентабельность %                   6.39
Рентабельность по EBITDA %                      6.74
Чистая рентабельность %                         3.30
Быстрая ликвидность                            -0.05
Коэффициент текущей ликвидности               

In [3]:
tickers = pd.read_csv(r'C:\Users\pc\Downloads\json_date\test.txt', header = None)
tickers
          

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0.088928,150.245098,32.075472,34.080717,57.377049,-23.080146,0.000000,-37.037037,-32.268796,-35.820896,0.000000,39.116719,0.000000,-44.896393,0.0,1.960784,0.0,20.000000,-44.000000,-49.517685,0.000000,-10.554665,sber
1,0.000000,-29.285015,80.000000,7.357860,43.750000,13.423981,0.000000,141.176471,104.518664,129.874776,0.000000,75.283447,0.000000,143.871866,0.0,7.692308,0.0,0.000000,-21.428571,-36.942675,-28.571429,54.986784,sber
2,-0.088849,-6.509695,34.920635,19.003115,7.246377,17.847737,0.000000,29.268293,0.384246,19.377432,0.000000,30.487279,0.000000,37.235865,0.0,-17.857143,0.0,-8.333333,-9.090909,-22.222222,0.000000,29.374308,sber
3,-0.088928,-24.444444,-22.352941,-15.445026,-26.351351,-6.086996,0.000000,7.924528,9.377990,16.883963,0.000000,8.724389,0.000000,8.947149,0.0,10.869565,0.0,0.000000,-10.000000,-6.493506,-60.000000,19.310206,sber
4,-0.044504,24.313725,27.272727,26.625387,17.431193,14.244908,0.000000,6.643357,-10.629921,-10.987172,0.000000,6.291793,0.000000,2.368220,0.0,-7.843137,0.0,0.000000,-11.111111,-30.555556,100.000000,1.210893,sber
5,0.000000,-18.296530,-25.595238,-0.488998,-28.125000,-27.512249,0.000000,-16.393443,-8.516887,-5.701754,0.000000,-42.922505,0.000000,-43.208955,0.0,-14.893617,0.0,0.000000,12.500000,24.000000,0.000000,0.849712,sber
6,0.000000,20.656371,7.200000,13.513514,3.260870,4.687939,0.000000,-10.588235,-11.289460,-10.099668,0.000000,51.402806,0.000000,26.872536,0.0,-2.500000,0.0,0.000000,0.000000,-3.225806,0.000000,-16.726240,sber
7,0.000000,8.800000,12.686567,8.874459,14.736842,26.102159,0.000000,2.631579,3.256936,2.143385,0.000000,7.081403,0.000000,64.733299,0.0,0.000000,0.0,-9.090909,11.111111,20.000000,0.000000,11.368833,sber
8,-0.044524,21.323529,17.880795,-15.109344,13.761468,36.619123,0.000000,-2.991453,-6.834112,-10.202605,0.000000,-11.464771,0.000000,-34.265954,0.0,2.564103,0.0,0.000000,0.000000,4.166667,0.000000,6.885211,sber
9,0.133630,-14.303030,2.247191,5.152225,3.225806,7.570283,0.000000,18.061674,20.564263,21.273167,0.000000,51.657941,0.000000,51.267336,0.0,7.500000,0.0,0.000000,40.000000,34.666667,0.000000,13.557356,sber


In [5]:
print(get_from_json_fund_params('gazp'))

    Всего выпущено обыкновенных акций  Цена/Прибыль  Цена/Продажи  \
0                            0.000000    -78.897137     -5.454545   
1                           -3.698869     -7.788945     13.461538   
2                            0.000000     10.081744    -23.728814   
3                            0.000000    -42.574257     -8.888889   
4                            6.823317    106.896552     82.926829   
5                            0.000000     53.750000    -20.000000   
6                            0.000000     52.168022     18.333333   
7                            0.000000      0.000000     -9.859155   
8                            0.000000    236.153161     25.000000   
9                            0.000000    -79.523179     -2.500000   
10                           0.000000    -19.016818     10.256410   
11                           0.000000     25.079872     24.418605   
12                          -0.000309      0.000000      0.000000   

    Цена/Движение денежных средст